In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [2]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,BatchNormalization,Dropout,AveragePooling2D
from keras.layers import Activation, Dropout, Flatten, Dense, GaussianNoise, GlobalMaxPooling2D
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import cv2
from sklearn import datasets
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import validation_curve

In [3]:
#Downloading dataset
!kaggle datasets download -d vuppalaadithyasairam/bone-fracture-detection-using-xrays

bone-fracture-detection-using-xrays.zip: Skipping, found more recently modified local copy (use --force to force download)


In [4]:
import zipfile
zip_ref = zipfile.ZipFile('/content/bone-fracture-detection-using-xrays.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [5]:
#Loading the images
x_train= keras.utils.image_dataset_from_directory(
    directory = '/content/archive (6)/train',
    labels='inferred',
    label_mode = 'int',
    batch_size=100,
    image_size=(256,256)
)

x_test= keras.utils.image_dataset_from_directory(
    directory = '/content/archive (6)/val',
    labels='inferred',
    label_mode = 'int',
    batch_size=100,
    image_size=(256,256)
)

Found 8863 files belonging to 2 classes.
Found 600 files belonging to 2 classes.


In [6]:
class_names = x_train.class_names
class_names

['fractured', 'not fractured']

In [9]:
# Normalizing
def process(image,label):
    image = tf.cast(image/255. ,tf.float32)
    return image,label

x_train = x_train.map(process)
x_test = x_test.map(process)

In [10]:
input_shape = (256, 256, 3)

In [11]:
#Model
model = Sequential()
model.add(Conv2D(32, (2, 2),activation='relu', input_shape=input_shape))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(32, (2, 2),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(64, (2, 2),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Flatten())

model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(1,activation='sigmoid'))

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 255, 255, 32)      416       
                                                                 
 batch_normalization (Batch  (None, 255, 255, 32)      128       
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 127, 127, 32)      0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 127, 127, 32)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 126, 126, 32)      4128      
                                                                 
 batch_normalization_1 (Bat  (None, 126, 126, 32)      1

In [13]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [14]:
history = model.fit(x_train,epochs=10, batch_size=100, validation_data=x_test)

Epoch 1/10
89/89 [==============================] - 1055s 12s/step - loss: 1.4677 - accuracy: 0.5635 - val_loss: 30.5254 - val_accuracy: 0.4000
Epoch 2/10
89/89 [==============================] - 1061s 12s/step - loss: 0.6262 - accuracy: 0.6028 - val_loss: 25.9065 - val_accuracy: 0.4000
Epoch 3/10
89/89 [==============================] - 1052s 12s/step - loss: 0.5976 - accuracy: 0.6409 - val_loss: 12.5752 - val_accuracy: 0.5333
Epoch 4/10
89/89 [==============================] - 1054s 12s/step - loss: 0.5620 - accuracy: 0.6694 - val_loss: 9.2154 - val_accuracy: 0.5833
Epoch 5/10
89/89 [==============================] - 1013s 11s/step - loss: 0.5387 - accuracy: 0.6911 - val_loss: 11.3421 - val_accuracy: 0.5450
Epoch 6/10
89/89 [==============================] - 1017s 11s/step - loss: 0.5275 - accuracy: 0.7063 - val_loss: 8.8051 - val_accuracy: 0.4450
Epoch 7/10
89/89 [==============================] - 994s 11s/step - loss: 0.4990 - accuracy: 0.7238 - val_loss: 7.9017 - val_accuracy: 0.3